In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003,0.741626,3.399873,640.534672,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,8.248824,417.592000
1,2004,0.742636,3.085211,650.203551,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,4.632078e+07,2.171425e+06,16.238609,3.976863e+07,8.178155,466.165000
2,2005,0.743635,2.624413,663.501150,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,4.700352e+07,2.187532e+06,16.294466,3.998653e+07,8.157501,479.481000
3,2006,0.744622,2.675575,712.049517,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,4.770379e+07,2.210487e+06,16.382367,4.026684e+07,8.070117,515.485000
4,2007,0.745596,3.084080,783.793548,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,4.835689e+07,2.229287e+06,16.460417,4.051516e+07,8.040811,632.929000
5,2008,0.746558,3.433682,831.774473,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,8.011861,814.771000
6,2009,0.747506,3.651735,863.877907,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,8.040524,693.327000
7,2010,0.748442,3.846003,916.677483,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,8.003862,813.149000
8,2011,0.749364,4.020724,958.130317,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,7.993144,923.248000
9,2012,0.750034,4.183487,956.016292,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,6.360445,1008.624000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
0,0.741626,3.399873,640.534672,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,8.248824
1,0.742636,3.085211,650.203551,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,4.632078e+07,2.171425e+06,16.238609,3.976863e+07,8.178155
2,0.743635,2.624413,663.501150,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,4.700352e+07,2.187532e+06,16.294466,3.998653e+07,8.157501
3,0.744622,2.675575,712.049517,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,4.770379e+07,2.210487e+06,16.382367,4.026684e+07,8.070117
4,0.745596,3.084080,783.793548,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,4.835689e+07,2.229287e+06,16.460417,4.051516e+07,8.040811
5,0.746558,3.433682,831.774473,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,8.011861
6,0.747506,3.651735,863.877907,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,8.040524
7,0.748442,3.846003,916.677483,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,8.003862
8,0.749364,4.020724,958.130317,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,7.993144
9,0.750034,4.183487,956.016292,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,6.360445


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      466.165000
1      479.481000
2      515.485000
3      632.929000
4      814.771000
5      693.327000
6      813.149000
7      923.248000
8     1008.624000
9     1028.608000
10     995.222143
11     951.041314
12     904.537543
13     870.341000
14     869.680000
15     902.144000
16    1038.045000
17    1092.056000
18    1152.060000
19            NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
0,-1.668612,-0.519867,-1.441613,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.582064,-2.203949,-1.714605,-1.673320,1.153654
1,-1.448585,-1.108989,-1.351083,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.407665,-1.729420,-1.593603,-1.506631,1.036910
2,-1.231160,-1.971711,-1.226579,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.235434,-1.337617,-1.402837,-1.318613,1.002789
3,-1.016392,-1.875923,-0.772023,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.058781,-0.779246,-1.102634,-1.076732,0.858432
4,-0.804333,-1.111106,-0.100288,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.894024,-0.321957,-0.836074,-0.862463,0.810019
5,-0.595037,-0.456568,0.348954,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.743785,0.191691,-0.585184,-0.681830,0.762193
6,-0.388558,-0.048322,0.649537,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.623067,0.650524,-0.476510,-0.569994,0.809545
7,-0.184948,0.315394,1.143896,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.252515,1.158607,-0.013403,-0.114566,0.748980
8,0.015739,0.642513,1.532016,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.086265,1.273468,0.368094,0.247582,0.731273
9,0.161537,0.947243,1.512223,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.391805,1.124521,0.623483,0.537011,-1.965926


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      466.165000
1      479.481000
2      515.485000
3      632.929000
4      814.771000
5      693.327000
6      813.149000
7      923.248000
8     1008.624000
9     1028.608000
10     995.222143
11     951.041314
12     904.537543
13     870.341000
14     869.680000
15     902.144000
16    1038.045000
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
17,0.566926,2.328712,0.362279,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.898913,-0.243065,0.698402,0.787591,-1.313749


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1092.056
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[12361479, 316997271, 124554141, 3316766024, 630900305, 3472103442, 1263116542, 3878077017, 1475472655, 373175504, 1286335487, 3623493033, 2310766674, 3073024582, 3791043386, 980393774, 1071754989, 446403032, 2977662170, 3991504943, 784254766, 166734797, 414965205, 1698039305, 3093188791, 3701190726, 3225644805, 2786264037, 3843511780, 1549831289, 3808123403, 168219377, 2043725695, 731551377, 887933306, 3787486526, 3078269802, 3053329926, 2679796760, 2789532058, 3620713234, 2535095204, 1380716809, 2932462880, 3431130692, 2528208072, 3097793316, 192196630, 4043472531, 2583995163, 3862234626, 2171266334, 2028250298, 2054037711, 842629600, 2171072538, 1972083154, 3833271118, 661528478, 1597264900, 4033630197, 1681717169, 2096503566, 3088393883, 20507644, 3619769446, 3154047363, 3660367434, 3503604578, 48104024, 1957885686, 169297864, 2736848424, 2225622509, 2503039256, 852295847, 2232639940, 1483703661, 2751003468, 3006287092, 383580860, 2176196166, 866419060, 4232689161, 1702547994, 1048

2023-10-01 14:47:08.737817: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5318.86669921875


Step: 43 ___________________________________________
val_loss: 5318.8701171875


Step: 44 ___________________________________________
val_loss: 3673.987060546875


Step: 45 ___________________________________________
val_loss: 5318.86669921875


Step: 46 ___________________________________________
val_loss: 5318.8671875


Step: 47 ___________________________________________
val_loss: 5318.8671875


Step: 48 ___________________________________________
val_loss: 4857.42041015625


Step: 49 ___________________________________________
val_loss: 5318.92822265625


Step: 50 ___________________________________________
val_loss: 5318.87548828125


Step: 51 ___________________________________________
val_loss: 5318.86669921875


Step: 52 ___________________________________________
val_loss: 5112.72509765625


Step: 53 ___________________________________________
val_loss: 5318.93408203125


Step: 54 ___________________________________________
val_loss: 5318.873046875

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 459ms/step - loss: 704131.0625 - val_loss: 868036.5625
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 690426.0625 - val_loss: 849063.2500
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 670177.7500 - val_loss: 1272019.8750
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 806693.4375 - val_loss: 339008.0938
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 608488.7500 - val_loss: 680453.3125
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 502074.2812 - val_loss: 254570.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 321270.1250 - val_loss: 191378.3750
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 294245.1875 - val_loss: 171935.1719
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 297255.7188 - val_loss: 185545.7500
Epoch 10

Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 138218.4375 - val_loss: 70118.3828
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 166224.9062 - val_loss: 232298.9219
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 119393.3672 - val_loss: 60510.3945
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 166531.7656 - val_loss: 371365.8438
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 175414.7031 - val_loss: 104380.8125
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 167682.6250 - val_loss: 170152.5625
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 168767.7812 - val_loss: 169043.8906
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 163658.8281 - val_loss: 227920.6719
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 188919.9844 - val_loss: 172604.8281
Epo

Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 62555.6602 - val_loss: 56841.9180
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 62187.1016 - val_loss: 57793.3945
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 71968.3281 - val_loss: 190898.3750
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 103095.3906 - val_loss: 51036.7227
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 74737.8750 - val_loss: 65471.2305
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 62578.0664 - val_loss: 61748.9336
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 65313.2656 - val_loss: 66284.0703
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 60164.4648 - val_loss: 81047.2734
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 61322.6992 - val_loss: 68218.5312
Epoch 15

1/1 [==============================] - 0s 15ms/step - loss: 33580.6328 - val_loss: 19477.9688
Epoch 224/10000
1/1 [==============================] - 0s 15ms/step - loss: 35362.6680 - val_loss: 20009.1875
Epoch 225/10000
1/1 [==============================] - 0s 15ms/step - loss: 32970.9727 - val_loss: 18576.4043
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 32414.2422 - val_loss: 17823.6230
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 32305.1133 - val_loss: 18407.5918
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 33262.3555 - val_loss: 17925.2656
Epoch 229/10000
1/1 [==============================] - 0s 15ms/step - loss: 33954.8789 - val_loss: 18575.6582
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 31592.5508 - val_loss: 17092.3145
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 31132.2168 - val_loss: 17185.5449
Epoch 232/10000
1/1 [=====

Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 23929.0078 - val_loss: 8308.4697
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 23857.6719 - val_loss: 8240.5010
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 23787.7500 - val_loss: 8173.6001
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 23719.2109 - val_loss: 8108.2417
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 23652.0352 - val_loss: 8044.2456
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 23586.1914 - val_loss: 7981.6343
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 23521.6543 - val_loss: 7920.3477
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 23458.3984 - val_loss: 7860.3911
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 23396.3945 - val_loss: 7801.7104
Epoch 307/10000
1/1

...

1/1 [==============================] - 0s 13ms/step - loss: 20221.9590 - val_loss: 5376.4614
Epoch 1347/10000
1/1 [==============================] - 0s 13ms/step - loss: 20590.8281 - val_loss: 5397.2344
Epoch 1348/10000
1/1 [==============================] - 0s 14ms/step - loss: 15865.9727 - val_loss: 5347.4658
Epoch 1349/10000
1/1 [==============================] - 0s 14ms/step - loss: 20773.5723 - val_loss: 5467.0884
Epoch 1350/10000
1/1 [==============================] - 0s 13ms/step - loss: 20592.4824 - val_loss: 5368.6450
Epoch 1351/10000
1/1 [==============================] - 0s 13ms/step - loss: 20573.7930 - val_loss: 5387.3984
Epoch 1352/10000
1/1 [==============================] - 0s 14ms/step - loss: 20158.8145 - val_loss: 5361.5625
Epoch 1353/10000
1/1 [==============================] - 0s 13ms/step - loss: 20564.9941 - val_loss: 5383.1484
Epoch 1354/10000
1/1 [==============================] - 0s 13ms/step - loss: 20558.8789 - val_loss: 5373.5933
Epoch 1355/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 15569.2100 - val_loss: 5563.9336
Epoch 1421/10000
1/1 [==============================] - 0s 13ms/step - loss: 18980.9902 - val_loss: 7155.5957
Epoch 1422/10000
1/1 [==============================] - 0s 13ms/step - loss: 19610.6895 - val_loss: 5691.0444
Epoch 1423/10000
1/1 [==============================] - 0s 13ms/step - loss: 17504.9648 - val_loss: 5322.7778
Epoch 1424/10000
1/1 [==============================] - 0s 13ms/step - loss: 17293.4082 - val_loss: 5365.9565
Epoch 1425/10000
1/1 [==============================] - 0s 13ms/step - loss: 16703.2402 - val_loss: 5401.1919
Epoch 1426/10000
1/1 [==============================] - 0s 13ms/step - loss: 17085.5391 - val_loss: 5352.5747
Epoch 1427/10000
1/1 [==============================] - 0s 14ms/step - loss: 17037.3066 - val_loss: 5365.5781
Epoch 1428/10000
1/1 [==============================] - 0s 14ms/step - loss: 17613.8691 - val_loss: 5353.0693
Epoch 1429/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 10482.1641 - val_loss: 5885.9531
Epoch 1495/10000
1/1 [==============================] - 0s 13ms/step - loss: 10266.5469 - val_loss: 6127.2681
Epoch 1496/10000
1/1 [==============================] - 0s 13ms/step - loss: 10227.4492 - val_loss: 6059.2642
Epoch 1497/10000
1/1 [==============================] - 0s 13ms/step - loss: 10200.1631 - val_loss: 6073.5317
Epoch 1498/10000
1/1 [==============================] - 0s 13ms/step - loss: 11111.2686 - val_loss: 5817.4106
Epoch 1499/10000
1/1 [==============================] - 0s 13ms/step - loss: 10181.0996 - val_loss: 6136.5552
Epoch 1500/10000
1/1 [==============================] - 0s 13ms/step - loss: 10135.4541 - val_loss: 6034.4414
Epoch 1501/10000
1/1 [==============================] - 0s 13ms/step - loss: 10106.3594 - val_loss: 6067.8984
Epoch 1502/10000
1/1 [==============================] - 0s 14ms/step - loss: 10081.7461 - val_loss: 6056.1021
Epoch 1503/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 11366.2305 - val_loss: 5468.7827
Epoch 1569/10000
1/1 [==============================] - 0s 13ms/step - loss: 11162.8301 - val_loss: 5621.8374
Epoch 1570/10000
1/1 [==============================] - 0s 14ms/step - loss: 11137.2480 - val_loss: 5576.4956
Epoch 1571/10000
1/1 [==============================] - 0s 14ms/step - loss: 11126.2764 - val_loss: 5587.3203
Epoch 1572/10000
1/1 [==============================] - 0s 14ms/step - loss: 11116.3955 - val_loss: 5583.8872
Epoch 1573/10000
1/1 [==============================] - 0s 14ms/step - loss: 11106.7002 - val_loss: 5584.1460
Epoch 1574/10000
1/1 [==============================] - 0s 14ms/step - loss: 11097.1240 - val_loss: 5583.4282
Epoch 1575/10000
1/1 [==============================] - 0s 14ms/step - loss: 11087.6602 - val_loss: 5582.9409
Epoch 1576/10000
1/1 [==============================] - 0s 14ms/step - loss: 11078.3066 - val_loss: 5582.3711
Epoch 1577/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 13715.9062 - val_loss: 5331.7905
Epoch 1643/10000
1/1 [==============================] - 0s 14ms/step - loss: 13875.5723 - val_loss: 5322.8828
Epoch 1644/10000
1/1 [==============================] - 0s 13ms/step - loss: 13647.4385 - val_loss: 5335.5508
Epoch 1645/10000
1/1 [==============================] - 0s 14ms/step - loss: 13612.7207 - val_loss: 5333.0356
Epoch 1646/10000
1/1 [==============================] - 0s 22ms/step - loss: 13580.6689 - val_loss: 5335.4419
Epoch 1646: early stopping


In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1092.056](test_target) - [[1054.0067]](prediction) = 38.04928613281254


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
0,-1.546922,0.426496,-1.180396,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.486090,-1.500496,-1.347217,-1.431223,1.847462
1,-1.093952,-0.341244,-1.083070,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.059395,-1.070923,-1.129572,-1.089335,1.004430
2,-0.646341,-1.465540,-0.949218,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.638006,-0.716239,-0.786442,-0.703703,0.758035
3,-0.204199,-1.340709,-0.460535,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.205797,-0.210768,-0.246470,-0.207592,-0.284392
4,0.232365,-0.344003,0.261633,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.197306,0.203199,0.232989,0.231883,-0.633996
5,0.663241,0.508987,0.744604,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.564886,0.668184,0.684262,0.602369,-0.979352
6,1.088319,1.041011,1.067754,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.860243,1.083547,0.879733,0.831749,-0.637417
7,1.507490,1.515004,1.599229,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.766854,1.543496,1.712719,1.765852,-1.074770


train_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
0,466.165
1,479.481
2,515.485
3,632.929
4,814.771
5,693.327
6,813.149
7,923.248


test_input:


...

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
17,0.566926,2.328712,0.362279,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.898913,-0.243065,0.698402,0.787591,-1.313749


test_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
17,1092.056


1/1 [==============================] - 0s 34ms/step
Error: 38.04928613281254




[811.052734375,
 937.06884765625,
 993.0635986328125,
 1011.70263671875,
 996.6561279296875,
 1027.934326171875,
 893.7736206054688,
 917.6531372070312,
 911.8599853515625,
 1054.0067138671875]

In [21]:
display(targets)
display(predictions)

[1008.624,
 1028.608,
 995.2221428571428,
 951.0413142857142,
 904.5375428571429,
 870.341,
 869.68,
 902.144,
 1038.045,
 1092.056]

[811.052734375,
 937.06884765625,
 993.0635986328125,
 1011.70263671875,
 996.6561279296875,
 1027.934326171875,
 893.7736206054688,
 917.6531372070312,
 911.8599853515625,
 1054.0067138671875]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

80.5479254464286

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.08338036477590247